In [1]:
!pip install openai

In [4]:
from getpass import getpass

In [5]:
import openai

In [ ]:
openai.api_key = getpass("AIP KEY ")
openai.api_base = "https://api.groq.com/openai/v1"

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··········


In [7]:
response = openai.ChatCompletion.create(
    model="llama3-8b-8192",
    messages=[{"role": "user", "content": "Hello, can you confirm my API key works?"}]
)

print(response.choices[0].message["content"])

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [8]:
from openai import OpenAI

client = OpenAI(api_key=openai.api_key, base_url="https://api.groq.com/openai/v1")

response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "user", "content": "Hello, can you confirm my API key works?"}]
)

print(response.choices[0].message.content)


BadRequestError: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

In [9]:
from openai import OpenAI

client = OpenAI(api_key=openai.api_key, base_url="https://api.groq.com/openai/v1")

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Hello, can you confirm my API key works?"}]
)

print(response.choices[0].message.content)


This conversation just started, and I'm a large language model, I don't have the ability to store or access your API key directly. I'm also not aware of any specific API key you might be referring to.

However, if you'd like to test your API key with me, you can try using it to authenticate with a specific API or service that I can interact with. Please let me know what kind of API key it is and how I can help you verify it. I can try to provide a test or example to help you check if your API key is working as expected.


In [10]:
conversation_history = []

def add_message(role, message):
    conversation_history.append({"role": role, "message": message})

add_message("user", "Hello, can you confirm my API key works?")
add_message("assistant", "Yes, your API key works perfectly!")

print(conversation_history)


[{'role': 'user', 'message': 'Hello, can you confirm my API key works?'}, {'role': 'assistant', 'message': 'Yes, your API key works perfectly!'}]


In [11]:
conversation_history = []

def add_message(role: str, content: str):
    assert role in ("user", "assistant", "system"), "role must be user|assistant|system"
    assert isinstance(content, str) and len(content) > 0, "content must be non-empty string"
    conversation_history.append({"role": role, "content": content})

In [12]:
def truncate_by_last_n(messages: list[dict], n: int | None):
    """
    Keep only the last n messages. If n is None, return all messages.
    Each list item is an OpenAI-format message: {"role": ..., "content": ...}
    """
    if n is None:
        return messages
    return messages[-n:]


In [13]:
def truncate_by_char_limit(messages: list[dict], max_chars: int | None):
    """
    Keep as many of the most recent messages as will fit within max_chars
    when summing len(content). If max_chars is None, return all messages.
    """
    if max_chars is None:
        return messages

    total = 0
    kept_reversed = []

    for msg in reversed(messages):
        c = len(msg.get("content", "") or "")
        if total + c > max_chars:
            break
        kept_reversed.append(msg)
        total += c

    return list(reversed(kept_reversed))

In [14]:
def apply_truncation(messages: list[dict], last_n: int | None = None, max_chars: int | None = None):
    """
    Apply character-limit first (to keep newest messages within size),
    then apply last-N to ensure a hard cap on message count if desired.
    """
    msgs = truncate_by_char_limit(messages, max_chars=max_chars)
    msgs = truncate_by_last_n(msgs, n=last_n)
    return msgs

In [15]:
def send_chat(
    client,
    model: str,
    user_text: str,
    last_n: int | None = None,
    max_chars: int | None = None,
    temperature: float = 0.3,
):
    """
    - Appends the user_text to the full conversation_history
    - Builds a truncated context using apply_truncation()
    - Calls chat.completions.create() on the Groq OpenAI-compatible endpoint
    - Appends the assistant reply back into full conversation_history
    - Returns the assistant reply text
    """
    add_message("user", user_text)

    context = apply_truncation(conversation_history, last_n=last_n, max_chars=max_chars)

    resp = client.chat.completions.create(
        model=model,
        messages=context,
        temperature=temperature,
    )
    reply = resp.choices.message.content
    add_message("assistant", reply)
    return reply

In [16]:
def summarize_history(client, model: str, messages_slice: list[dict], max_words: int = 120) -> str:
    """
    Calls the model to summarize the provided messages_slice.
    Returns a concise summary string with key intents, decisions, and open questions.
    """
    system_prompt = (
        "You are a conversation summarizer. "
        f"Summarize the chat succinctly in <= {max_words} words. "
        "Capture: user goals, assistant guidance, key decisions, and open questions."
    )

    # Prepare a self-contained input: system + the slice as a compact text block
    # We stringify the slice so the model sees exactly what to summarize.
    slice_text = "\n".join(f"{m['role']}: {m.get('content','')}" for m in messages_slice)

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Summarize the following dialogue:\n\n{slice_text}"},
        ],
        temperature=0.2,
    )

    return resp.choices.message.content.strip()


In [17]:

def compact_history_with_summary(
    client,
    model: str,
    keep_last: int = 6,
    max_words: int = 120,
):
    """
    - Summarizes all but the most recent `keep_last` messages.
    - Replaces that older block with a single assistant summary message.
    - Leaves the last `keep_last` messages verbatim.
    """
    global conversation_history
    if len(conversation_history) <= keep_last:
        return  # nothing to compact

    older = conversation_history[:-keep_last]
    recent = conversation_history[-keep_last:]

    summary_text = summarize_history(client, model, older, max_words=max_words)

    summary_msg = {
        "role": "assistant",
        "content": f"(Summary of earlier context) {summary_text}"
    }

    conversation_history = [summary_msg] + recent


In [18]:
turn_count = 0

def send_chat_with_periodic_summary(
    client,
    model: str,
    user_text: str,
    k: int = 3,
    keep_last: int = 6,
    last_n: int | None = None,
    max_chars: int | None = None,
    temperature: float = 0.3,
    summary_max_words: int = 120,
):
    """
    - Sends a chat message via send_chat (which applies truncation).
    - After every k-th user turn, compacts earlier history into one summary,
      keeping the most recent `keep_last` messages verbatim.
    """
    global turn_count

    reply = send_chat(
        client=client,
        model=model,
        user_text=user_text,
        last_n=last_n,
        max_chars=max_chars,
        temperature=temperature,
    )

    turn_count += 1
    if k is not None and k > 0 and (turn_count % k == 0):
        compact_history_with_summary(
            client=client,
            model=model,
            keep_last=keep_last,
            max_words=summary_max_words,
        )

    return reply


In [23]:
conversation_history.clear()
turn_count = 0

model_id = "llama-3.1-8b-instant"


inputs = [
    "Hi, I want help planning a study schedule for Oracle Gen AI exam.",
    "I can study 2 hours daily. Prefer mornings.",
    "Please suggest a 2-week plan with milestones.",
    "Also add quick review checkpoints every 3 days.",
]

for i, text in enumerate(inputs, start=1):
    print(f"\n--- User turn {i} ---")
    reply = send_chat_with_periodic_summary(
        client=client,
        model=model_id,
        user_text=text,
        k=3,
        keep_last=6,
        last_n=12,
        max_chars=4000,
        summary_max_words=80,
    )
    print("Assistant:", reply)


    roles = [m["role"] for m in conversation_history]
    print("History size:", len(conversation_history), "| roles:", roles)

print("\n=== Final conversation history ===")
for m in conversation_history:
    print(f"{m['role']}: {m['content'][:120]}{'...' if len(m['content'])>120 else ''}")



--- User turn 1 ---


AttributeError: 'list' object has no attribute 'message'

In [25]:

def send_chat(
    client,
    model: str,
    user_text: str,
    last_n: int | None = None,
    max_chars: int | None = None,
    temperature: float = 0.3,
):
    add_message("user", user_text)

    context = apply_truncation(conversation_history, last_n=last_n, max_chars=max_chars)

    resp = client.chat.completions.create(
        model=model,
        messages=context,
        temperature=temperature,
    )

    reply = resp.choices[0].message.content  # <-- fixed indexing
    add_message("assistant", reply)
    return reply

In [26]:
conversation_history.clear()
turn_count = 0

model_id = "llama-3.1-8b-instant"  # or "llama-3.3-70b-versatile"

inputs = [
    "We need to design a small FAQ bot for a college site.",
    "Data sources are a CSV of Q&A and a PDF brochure.",
    "Suggest a plan with steps and a minimal tech stack.",
    "Add a quick timeline for a 2-week delivery.",
]

for i, text in enumerate(inputs, start=1):
    print(f"\n--- User turn {i} ---")
    reply = send_chat_with_periodic_summary(
        client=client,
        model=model_id,
        user_text=text,
        k=3,            # summarize after every 3rd user turn
        keep_last=6,    # keep recent context verbatim
        last_n=12,      # cap number of messages sent
        max_chars=4000, # cap total characters sent
        summary_max_words=80,
    )
    print("Assistant:", reply)

print("\n=== Final conversation history (roles only) ===")
print([m["role"] for m in conversation_history])

print("\n=== Final conversation history (preview) ===")
for m in conversation_history:
    print(f"{m['role']}: {m['content'][:140]}{'...' if len(m['content'])>140 else ''}")


--- User turn 1 ---
Assistant: Here's a basic design for a small FAQ bot for a college site:

**Bot Name:** CollegePal

**Functionality:**

1. **User Input:** The bot will accept user input in the form of questions or keywords related to college life.
2. **Question Matching:** The bot will match the user's input with a predefined list of FAQs.
3. **Answer Retrieval:** The bot will retrieve the corresponding answer from the FAQ database.
4. **Response Generation:** The bot will generate a response to the user, including the answer and any relevant additional information.

**Design Requirements:**

1. **Natural Language Processing (NLP):** The bot will use NLP to understand the user's input and match it with the FAQs.
2. **FAQ Database:** The bot will have a database of FAQs, which will be populated with common questions and answers related to college life.
3. **User Interface:** The bot will have a simple user interface, such as a chat window or a web form, where users can input their 

In [27]:
extract_profile_tool = [{
    "type": "function",
    "function": {
        "name": "extract_profile",
        "description": "Extract a simple profile from free text.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Full name if present; otherwise empty string."
                },
                "email": {
                    "type": "string",
                    "description": "Email address if present; otherwise empty string."
                },
                "phone": {
                    "type": "string",
                    "description": "Phone number if present; include country/area code if available."
                },
                "location": {
                    "type": "string",
                    "description": "City and state/region/country if present; otherwise empty string."
                },
                "age": {
                    "type": "integer",
                    "description": "Age in years if present; otherwise 0."
                }
            },
            "required": ["name", "email", "phone", "location", "age"],
            "additionalProperties": False
        }
    }
}]


In [30]:
def run_extraction_with_tools(client, model: str, user_text: str):
    """
    Sends user_text with the extract_profile_tool available.
    Returns the raw response so the caller can parse tool_calls.
    """
    resp = client.chat.completions.create(
        model=model,  # e.g., "llama-3.1-8b-instant" or "llama-3.3-70b-versatile"
        messages=[
            {"role": "system", "content": "When relevant, extract profile fields via the provided function."},
            {"role": "user", "content": user_text},
        ],
        tools=extract_profile_tool,        # from Step 5a
        tool_choice="auto",               # let the model decide
        temperature=0.2,
    )
    return resp


In [33]:

import json

def parse_extraction_response(resp):
    """
    Extracts the first tool call's JSON arguments if present,
    returns a dict with fields: name, email, phone, location, age.
    Falls back to empty/defaults if no tool call was made.
    """
    result = {"name": "", "email": "", "phone": "", "location": "", "age": 0}

    if not getattr(resp, "choices", None):
        return result

    choice0 = resp.choices[0] # Access the first choice
    tool_calls = getattr(choice0.message, "tool_calls", None)

    if not tool_calls:
        return result
    call = tool_calls[0]
    fn = getattr(call, "function", None)
    if not fn or not getattr(fn, "arguments", None):
        return result

    # Parse JSON arguments string into a dict
    try:
        args = json.loads(fn.arguments)
    except Exception:
        return result

    name = str(args.get("name", "") or "")
    email = str(args.get("email", "") or "")
    phone = str(args.get("phone", "") or "")
    location = str(args.get("location", "") or "")

    age_raw = args.get("age", 0)
    try:
        age = int(age_raw)
    except Exception:
        age = 0

    result.update({"name": name, "email": email, "phone": phone, "location": location, "age": age})
    return result



In [34]:

samples = [
    "I'm Riya Patel, 21, from Ahmedabad. Email: riya.p@example.com, phone +91 98765 43210.",
    "This is Arjun M., based in Surat. Contact me at arjun.m@demo.in. Age twenty is fine. Call 079-22223333.",
    "Name: (unknown). Reach at support@example.org, age 34, location Rajkot, phone missing.",
]

model_id = "llama-3.1-8b-instant"  # or "llama-3.3-70b-versatile"

for i, text in enumerate(samples, start=1):
    print(f"\n--- Sample {i} ---")
    raw = run_extraction_with_tools(client, model_id, text)
    data = parse_extraction_response(raw)
    print("Input:", text)
    print("Parsed:", data)


--- Sample 1 ---
Input: I'm Riya Patel, 21, from Ahmedabad. Email: riya.p@example.com, phone +91 98765 43210.
Parsed: {'name': 'Riya Patel', 'email': 'riya.p@example.com', 'phone': '+91 98765 43210', 'location': 'Ahmedabad', 'age': 21}

--- Sample 2 ---
Input: This is Arjun M., based in Surat. Contact me at arjun.m@demo.in. Age twenty is fine. Call 079-22223333.
Parsed: {'name': 'Arjun M.', 'email': 'arjun.m@demo.in', 'phone': '079-22223333', 'location': 'Surat', 'age': 20}

--- Sample 3 ---
Input: Name: (unknown). Reach at support@example.org, age 34, location Rajkot, phone missing.
Parsed: {'name': '(unknown)', 'email': 'support@example.org', 'phone': '', 'location': 'Rajkot', 'age': 34}


In [35]:

import json

def extract_with_json_mode(client, model: str, user_text: str):
    """
    Forces the model to return strict JSON conforming to the schema.
    No function-calling/tools are used here.
    """
    schema = {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string"},
            "phone": {"type": "string"},
            "location": {"type": "string"},
            "age": {"type": "integer"}
        },
        "required": ["name", "email", "phone", "location", "age"],
        "additionalProperties": False
    }

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Extract the requested fields and return ONLY valid JSON matching the provided schema."},
            {"role": "user", "content": user_text},
        ],
        response_format={  # Groq supports OpenAI-compatible structured outputs
            "type": "json_schema",
            "json_schema": {
                "name": "profile_schema",
                "schema": schema,
                "strict": True  # enforce strict conformance
            }
        },
        temperature=0.2,
    )

    # The text is guaranteed to be valid JSON under strict mode
    raw_json = resp.choices.message.content
    data = json.loads(raw_json)
    return data

In [38]:
def extract_with_json_mode(client, model: str, user_text: str):
    import json
    schema_hint = "Return ONLY a JSON object with keys: name, email, phone, location, age. Age must be an integer."
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": schema_hint},
            {"role": "user", "content": user_text},
        ],
        response_format={"type": "json_object"},
        temperature=0.2,
    )
    raw = resp.choices[0].message.content # Access the message content from the first choice
    data = json.loads(raw)
    # coerce/validate fields as before
    data.setdefault("name",""); data.setdefault("email",""); data.setdefault("phone",""); data.setdefault("location","");
    try: data["age"] = int(data.get("age", 0))
    except Exception: data["age"] = 0
    return data

In [39]:
def compare_extraction_paths(client, model: str, text: str):
    print("\n=== Input ===")
    print(text)

    # Path 1: Function calling (tools)
    resp_tools = run_extraction_with_tools(client, model, text)
    parsed_tools = parse_extraction_response(resp_tools)

    # Path 2: Strict JSON mode (no tools)
    parsed_json_mode = extract_with_json_mode(client, model, text)  # Option A1

    print("\n--- Function Calling (tools) ---")
    print(parsed_tools)

    print("\n--- Strict JSON Mode ---")
    print(parsed_json_mode)

# --- Example usage ---
sample = "Hi, I'm Riya Patel, 21, based in Ahmedabad. Email: riya.p@example.com, phone +91-98765-43210."
compare_extraction_paths(client, "llama-3.1-8b-instant", sample)


=== Input ===
Hi, I'm Riya Patel, 21, based in Ahmedabad. Email: riya.p@example.com, phone +91-98765-43210.

--- Function Calling (tools) ---
{'name': '', 'email': '', 'phone': '', 'location': '', 'age': 0}

--- Strict JSON Mode ---
{'name': 'Riya Patel', 'email': 'riya.p@example.com', 'phone': '+91-98765-43210', 'location': 'Ahmedabad', 'age': 21}
